<a href="https://colab.research.google.com/github/smf-9000/images-through-ML/blob/main/objectDetectionApiTF%5Bexample%5D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>